In [28]:
import mysql.connector as connector
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

In [29]:
# Establish a connection pool named 'pool_b' with a maximum size of 2.
# Connection pooling allows multiple database connections without needing to re-establish a new connection each time.

db_config ={
    'database' : 'little_lemon_db', 
    'user' : 'root', 
    'password' : 'root'
}

try:
    pool = MySQLConnectionPool(
        pool_name='pool_b', 
        pool_size=2, 
        **db_config
    )
    print('Connection pool named {} created with size {}'.format(pool.pool_name, pool.pool_size))
except Error as err:
    print('Error code {}. \nError message'.format(err.errno, err.msg))


Connection pool named pool_b created with size 2


In [30]:
# Using the first pooled connection (conn_1) to add a new booking entry to the 'Bookings' table.

conn_1 = pool.get_connection()
cursor_1 = conn_1.cursor()

booking_1 = """
INSERT INTO Bookings
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(8, 'Anees', 'Java', '18:00:00', 6)
"""

cursor_1.execute(booking_1)
conn_1.commit()

print('New booking added on Bookings table')

New booking added on Bookings table


In [31]:
# Using the second pooled connection (conn_2) to add another booking entry to the 'Bookings' table.

conn_2 = pool.get_connection()
cursor_2 = conn_2.cursor()

booking_2 = """
INSERT INTO Bookings
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(5, 'Bald','Vin','19:00:00',6)
"""

cursor_2.execute(booking_2)
conn_2.commit()
print('New booking added on Bookings table')

New booking added on Bookings table


In [32]:
# Attempt to get a third connection from the pool.
# If the pool is exhausted (since the pool size is set to 2), add a new connection to the pool and then fetch it.

try:
    conn_3 = pool.get_connection()
    print('Guest is connected')
except:
    print('Adding new connection to the pool')

    connection = connector.connect(user='root', password='root')
    pool.add_connection(cnx=connection)
    print('New connection added to the pool')

    conn_3 = pool.get_connection()

Adding new connection to the pool
New connection added to the pool


In [33]:
cursor_3 = conn_3.cursor()

booking_3 = """
INSERT INTO Bookings
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(12, 'Jay','Kon','19:30:00',6)
"""

cursor_3.execute(booking_3)
conn_3.commit()
print('New booking added on Booking table')

New booking added on Booking table


In [34]:
# Close all the pooled connections, returning them to the connection pool.

for conn in [conn_1, conn_2, conn_3]:
    try:
        conn.close()
        print('Connection returned to the pool')
    except Error as err:
        print('Connection can not return to the pool. \n{}'.format(err.msg))

Connection returned to the pool
Connection returned to the pool
Connection can not return to the pool. 
Failed adding connection; queue is full


In [35]:
connection = pool.get_connection()
cursor = connection.cursor()

cursor.execute(
    """
    SELECT
    Name, EmployeeID
    
    FROM
    Employees
    
    WHERE
    Role = 'Manager'
    """
)

results = cursor.fetchall()
columns = cursor.column_names

print(columns)
for i in results:
    print(i)

('Name', 'EmployeeID')
('Mario Gollini', 1)


In [36]:
cursor.execute(
    """
    SELECT
    Name, EmployeeID
    
    FROM
    Employees
    
    ORDER BY
    Annual_Salary DESC LIMIT 1
    """
)

results = cursor.fetchall()
columns = cursor.column_names

print(columns)
for i in results:
    print(i)

('Name', 'EmployeeID')
('Mario Gollini', 1)


In [37]:
cursor.execute(
    """
    SELECT
    COUNT(BookingID) AS No_Booking_18_20
    
    FROM
    Bookings
    
    WHERE
    BookingSlot BETWEEN '18:00:00' AND '20:00:00'
    """
)

results = cursor.fetchall()
columns = cursor.column_names

print(columns)
for i in results:
    print(i)

('No_Booking_18_20',)
(7,)


In [38]:
# Execute an SQL query to retrieve guests waiting to be seated.
# The query joins the 'Bookings' and 'Employees' tables and filters by employee roles.

cursor.execute(
    """
    SELECT
    Bookings.BookingID AS ID,
    CONCAT(Bookings.GuestFirstName, ' ', Bookings.GuestLastName) AS GuestName, 
    Employees.Role AS Employee

    FROM
    Bookings

    LEFT JOIN
    Employees
    ON
    Employees.EmployeeID = Bookings.EmployeeID

    WHERE
    Employees.Role = 'Receptionist'

    ORDER BY
    BookingSlot DESC
    """
)

print('Guest waiting to be seated:\n')

results = cursor.fetchall()
columns = cursor.column_names

print(columns)
for i in results:
    print(i)

Guest waiting to be seated:

('ID', 'GuestName', 'Employee')
(9, 'Jay Kon', 'Receptionist')
(8, 'Bald Vin', 'Receptionist')
(7, 'Anees Java', 'Receptionist')


In [39]:
cursor.execute(
    """
    DROP PROCEDURE IF EXISTS
    BasicSalesReport
    """
)
# First, drop the 'BasicSalesReport' stored procedure if it already exists.
# Then, define and create a stored procedure named 'BasicSalesReport' that calculates basic sales metrics.
# The procedure fetches the total, average, minimum, and maximum bill amounts from the 'Orders' table.

basic_sales_report = """
CREATE PROCEDURE
BasicSalesReport()

BEGIN

SELECT
SUM(BillAmount) AS Total_sale, 
AVG(BillAmount) AS Avg_sale, 
MIN(BillAmount) AS Min_bill, 
MAX(BillAmount) AS Max_bill

FROM
Orders;

END
"""

cursor.execute(basic_sales_report)
cursor.callproc('BasicSalesReport')

results = next(cursor.stored_results())
db = results.fetchall()

for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

print('Sales report:\n')

for r in db:
    print("\t",columns[0],":",r[0])
    print("\t",columns[1],":",r[1])
    print("\t",columns[2],":",r[2])
    print("\t",columns[3],":",r[3])

connection.close()

Sales report:

	 Total_sale : 243
	 Avg_sale : 48.6000
	 Min_bill : 37
	 Max_bill : 86


In [40]:
# Execute an SQL query that joins the 'Bookings' and 'Employees' tables on EmployeeID.
# The query retrieves booking slots, guest names, employee names, and their roles.
# Results are ordered by booking slots in ascending order.

connection = pool.get_connection()
cursor = connection.cursor(buffered=True)

query = """
SELECT
Bookings.BookingSlot AS BookingSlot, 
CONCAT(Bookings.GuestFirstName, ' ', Bookings.GuestLastName) AS GuestName, 
Employees.Name AS EmployeeName, 
Employees.Role AS EmployeeRole

FROM
Bookings

INNER JOIN
Employees
ON Employees.EmployeeID = Bookings.EmployeeID

ORDER BY
Bookings.BookingSlot ASC
"""

cursor.execute(query)
results = cursor.fetchmany(size = 3)

columns = cursor.column_names

print(columns)
for i in results:
    print(i)

connection.close()

('BookingSlot', 'GuestName', 'EmployeeName', 'EmployeeRole')
(datetime.timedelta(seconds=54000), 'Vanessa McCarthy', 'Giorgos Dioudis', 'Head Chef')
(datetime.timedelta(seconds=63000), 'Marcos Romero', 'Fatma Kaya', 'Assistant Chef')
(datetime.timedelta(seconds=64800), 'Anees Java', 'John Millar', 'Receptionist')
